# Run the following code if working in collab

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cimat-ris/TF-PathPred.git clonedrep

import os
os.chdir("./clonedrep")

!pip install pykalman

# Testing the Transformer

This notebook enables the testing of the prediction of the transformer model for the datasets of ETH and UCY. It also gives a visualization of such predictions, and a visualization of the attention weight matrices

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import tensorflow as tf
from test_TF import test_model, print_sol
import matplotlib.pyplot as plt
import numpy as np
import os

from tools.trajectories import traj_to_real_coordinates

# Generate testing information

Run the cell bellow if you wish to generate the testing information. If you only want to visualize, go on to the next section.

In [2]:
path = "./"
#choose a dataset
test_dataset = "ETH-Univ"

test_path = path + f"generated_data/testing_data/{test_dataset}/"

try: os.mkdir(path+"generated_data/testing_data")
except: pass
try: os.mkdir(path+f"generated_data/testing_data/{test_dataset}")
except: pass

ade,fde,weights,trajs,transformer = test_model([test_dataset],path,5)

ade,fde = np.array(ade), np.array(fde)

np.save(test_path+"ade.npy", ade)
np.save(test_path+"fde.npy", fde)
np.save(test_path+"inp.npy", trajs[0])
np.save(test_path+"tar.npy", trajs[1])
np.save(test_path+"pred.npy", trajs[2])

Loading trajlets from:  ./generated_data/trajlets\ETH-Univ-trl.npy
(294, 7, 2)
(294, 12, 2)
./generated_data/checkpoints/train/ETH-Univ
Latest checkpoint restored!! ./generated_data/checkpoints/train/ETH-Univ\ckpt-17
Calculating predictions
ADE: 6.009637 FDE: 18.043371


In [3]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Encoder)            multiple                  149760    
_________________________________________________________________
decoder (Decoder)            multiple                  299136    
_________________________________________________________________
multi_modal (multi_modal)    multiple                  258       
Total params: 449,154
Trainable params: 449,154
Non-trainable params: 0
_________________________________________________________________


# Visualizing solution

Just run the cell bellow changing the name of the dataset you wish to visualize and change the value in the input box to visualize different trajectories. The red one is the observed trajectory, the blue one is the target, and the green ones are the possible outcomes predicted. If you wish to visualize a mode in particular just add a comma and the number of mode to visualize, with no spaces. e.g. "0,0"

In [10]:
pred[14][0]

array([[10.37353611,  2.88749385],
       [-2.58347797, -0.01147409],
       [-0.70064276, -0.48217607],
       [-0.23134898, -0.07329389],
       [-0.21743804,  0.13063127],
       [-0.10618126, -0.0674639 ],
       [ 0.09643086, -0.58270329],
       [ 0.14936332, -1.00543475],
       [-0.0643439 , -1.13872981],
       [-0.33580238, -1.09564412],
       [-0.49288875, -1.00369215],
       [-0.53886431, -0.90222126],
       [-0.53009629, -0.81128484]])

In [24]:
np.linalg.norm(pred[9][0]-tar[9])**2

3111.5274509651576

In [21]:
np.linalg.norm(pred[9][0]-tar[9])**2/tar[9].shape[0]

239.3482654588583

In [14]:
tar[9]

array([[ 6.899096 ,  3.4926176],
       [ 8.167761 ,  3.8916903],
       [ 9.171082 ,  4.6193113],
       [10.606197 ,  4.927866 ],
       [11.678649 ,  5.3118052],
       [12.873566 ,  5.837174 ],
       [14.0700035,  6.4906344],
       [15.28206  ,  7.1526256],
       [16.523434 ,  7.700064 ],
       [17.637083 ,  8.360063 ],
       [18.751009 ,  9.152999 ],
       [19.905037 ,  9.704154 ],
       [20.93805  , 10.241766 ]], dtype=float32)

In [3]:
path = "./"
#choose a dataset
test_dataset = "ETH-univ"

test_path = path + f"generated_data/testing_data/{test_dataset}/"
reference_path = path + f"datasets/ETH/seq_eth/reference.png"
H_path = path + f"datasets/ETH/seq_eth/H.txt"

H = np.loadtxt(H_path)
H = np.linalg.inv(H)
img = plt.imread(reference_path)

ade = np.load(test_path+"ade.npy")
fde = np.load(test_path+"fde.npy")
inp = np.load(test_path+"inp.npy")
tar = np.load(test_path+"tar.npy")
pred = np.load(test_path+"pred.npy")

print("General: ADE:", np.mean(ade),"FDE:", np.mean(fde))

def f(x):
    x = x.split(',')
    if len(x) == 1:
        try:
            x = int(x[0])
        except:
            print("Error: Input was not an integer")
            return

        if x >= len(ade):
            print("There aren't that many trajectories")
            return
        #print_sol(inp[x],tar[x],pred[x],None)
        a = traj_to_real_coordinates(inp[x],H)
        b = traj_to_real_coordinates(tar[x],H)
        c = traj_to_real_coordinates(pred[x],H)
        print_sol(a,b,c,img)
        
        print(f"trajectory {x}:","ADE:", ade[x],"FDE:", fde[x])
        
    elif len(x) == 2:
        try:
            k = int(x[1])
            x = int(x[0])
        except:
            print("Error: Inputs were not integers")
            return

        if x >= len(ade):
            print("There aren't that many trajectories")
            return
        #print_sol(inp[x],tar[x],pred[x][k:(k+1)],img)
        a = inp[x]#traj_to_real_coordinates(inp[x],H)
        b = tar[x]#traj_to_real_coordinates(tar[x],H)
        c = pred[x][k]#traj_to_real_coordinates(pred[x][k],H)
        print_sol(a,b,None,None)#np.array([c]),None)
        print(f"trajectory {x}:","ADE:", ade[x],"FDE:", fde[x])
    else:
        print("the format is not valid")

interact(f, x="0");

General: ADE: 6.009637 FDE: 18.043371


interactive(children=(Text(value='0', description='x'), Output()), _dom_classes=('widget-interact',))

In [5]:
inp.shape

(294, 19, 2)